### plot_walk_forward_v1

In [ ]:
# --- COMPLETE CONTEXT AND TEST SCRIPT (v2 - ALL FUNCTIONS INCLUDED) ---
# This script contains the final refactored code and a self-contained test case.
# Running this single cell will execute the test.

import pandas as pd
import plotly.graph_objects as go
from datetime import datetime, date
import numpy as np
import ipywidgets as widgets
from IPython.display import display, Markdown
import pprint
import io

import pandas as pd
import numpy as np

from datetime import datetime, date
from IPython.display import display, Markdown
from pathlib import Path

# --- A. HELPER FUNCTIONS (Shared across tools) ---

def calculate_gain(price_series: pd.Series):
    if price_series.dropna().shape[0] < 2: return np.nan
    return (price_series.ffill().iloc[-1] / price_series.bfill().iloc[0]) - 1

def calculate_sharpe(return_series: pd.Series):
    if return_series.dropna().shape[0] < 2: return np.nan
    std_dev = return_series.std()
    if std_dev > 0 and std_dev != np.inf:
        return (return_series.mean() / std_dev) * np.sqrt(252)
    return np.nan

# --- B. THE CORE CALCULATION ENGINE (Headless, No UI) ---

def run_walk_forward_step(df_close_full, df_high_full, df_low_full,
                          start_date, calc_period, fwd_period,
                          metric, rank_start, rank_end, benchmark_ticker):
    min_date_available = df_close_full.index.min()
    max_date_available = df_close_full.index.max()
    safe_start_date = max(start_date, min_date_available)
    safe_calc_end_date = min(start_date + calc_period, max_date_available)
    safe_viz_end_date = min(safe_calc_end_date + fwd_period, max_date_available)
    if safe_start_date >= safe_calc_end_date: return {'error': "Invalid date range."}
    calc_close_raw = df_close_full.loc[safe_start_date:safe_calc_end_date]
    calc_close = calc_close_raw.dropna(axis=1, how='all')
    if calc_close.shape[1] == 0 or len(calc_close) < 2: return {'error': "Not enough data in calc period."}

    metric_values = {}
    first_prices = calc_close.bfill().iloc[0]; last_prices = calc_close.ffill().iloc[-1]
    metric_values['Price'] = (last_prices / first_prices).dropna()
    daily_returns = calc_close.bfill().ffill().pct_change()
    mean_returns, std_returns = daily_returns.mean(), daily_returns.std()
    metric_values['Sharpe'] = (mean_returns / std_returns * np.sqrt(252)).fillna(0)
    valid_tickers = calc_close.columns
    calc_high = df_high_full[valid_tickers].loc[safe_start_date:safe_calc_end_date]
    calc_low = df_low_full[valid_tickers].loc[safe_start_date:safe_calc_end_date]
    tr = np.maximum(calc_high - calc_low, abs(calc_high - df_close_full[valid_tickers].shift(1)), abs(calc_low - df_close_full[valid_tickers].shift(1)))
    atr = tr.ewm(alpha=1/14, adjust=False).mean()
    atrp = (atr / calc_close).mean()
    metric_values['Sharpe (ATR)'] = (mean_returns / atrp).fillna(0)
    
    sorted_tickers = metric_values[metric].sort_values(ascending=False)
    tickers_to_display = sorted_tickers.index[rank_start-1:rank_end].tolist()
    if not tickers_to_display: return {'error': "No tickers found for the selected rank."}
        
    normalized_plot_data = df_close_full[tickers_to_display].loc[safe_start_date:safe_viz_end_date]
    normalized_plot_data = normalized_plot_data.div(normalized_plot_data.bfill().iloc[0])
    actual_calc_end_ts = calc_close.index.max()
    portfolio_series = normalized_plot_data.mean(axis=1)
    portfolio_return_series = portfolio_series.pct_change()
    benchmark_price_series = df_close_full.get(benchmark_ticker)
    benchmark_return_series = benchmark_price_series.loc[safe_start_date:safe_viz_end_date].bfill().ffill().pct_change() if benchmark_price_series is not None else pd.Series(dtype='float64')

    perf_data = {}
    perf_data['calc_p_gain'] = calculate_gain(portfolio_series.loc[:actual_calc_end_ts])
    perf_data['fwd_p_gain'] = calculate_gain(portfolio_series.loc[actual_calc_end_ts:])
    perf_data['full_p_gain'] = calculate_gain(portfolio_series)
    perf_data['calc_p_sharpe'] = calculate_sharpe(portfolio_return_series.loc[:actual_calc_end_ts])
    perf_data['fwd_p_sharpe'] = calculate_sharpe(portfolio_return_series.loc[actual_calc_end_ts:])
    perf_data['full_p_sharpe'] = calculate_sharpe(portfolio_return_series)
    perf_data['calc_b_gain'] = calculate_gain(benchmark_price_series.loc[safe_start_date:actual_calc_end_ts]) if benchmark_price_series is not None else np.nan
    perf_data['fwd_b_gain'] = calculate_gain(benchmark_price_series.loc[actual_calc_end_ts:safe_viz_end_date]) if benchmark_price_series is not None else np.nan
    perf_data['full_b_gain'] = calculate_gain(benchmark_price_series.loc[safe_start_date:safe_viz_end_date]) if benchmark_price_series is not None else np.nan
    perf_data['calc_b_sharpe'] = calculate_sharpe(benchmark_return_series.loc[:actual_calc_end_ts])
    perf_data['fwd_b_sharpe'] = calculate_sharpe(benchmark_return_series.loc[actual_calc_end_ts:])
    perf_data['full_b_sharpe'] = calculate_sharpe(benchmark_return_series)

    calc_end_prices = calc_close.ffill().iloc[-1]
    fwd_close_slice = df_close_full.loc[actual_calc_end_ts:safe_viz_end_date]
    viz_end_prices = fwd_close_slice.ffill().iloc[-1] if not fwd_close_slice.empty and len(fwd_close_slice) >= 2 else calc_end_prices
    calc_gains = (calc_end_prices / calc_close.bfill().iloc[0]) - 1
    fwd_gains = (viz_end_prices / calc_end_prices) - 1
    results_df = pd.DataFrame({'Rank': range(rank_start, rank_start + len(tickers_to_display)), 'Metric': metric, 'MetricValue': sorted_tickers.loc[tickers_to_display].values, 'CalcPrice': calc_end_prices.loc[tickers_to_display], 'CalcGain': calc_gains.loc[tickers_to_display], 'FwdGain': fwd_gains.loc[tickers_to_display]}, index=pd.Index(tickers_to_display, name='Ticker'))
    if benchmark_price_series is not None and benchmark_ticker in calc_close.columns:
        benchmark_df_row = pd.DataFrame({'Rank': np.nan, 'Metric': metric, 'MetricValue': metric_values[metric].get(benchmark_ticker, np.nan), 'CalcPrice': calc_end_prices[benchmark_ticker], 'CalcGain': calc_gains[benchmark_ticker], 'FwdGain': fwd_gains[benchmark_ticker]}, index=pd.Index([f"{benchmark_ticker} (BM)"], name='Ticker'))
        results_df = pd.concat([results_df, benchmark_df_row])
    
    return { 'tickers_to_display': tickers_to_display, 'normalized_plot_data': normalized_plot_data, 'portfolio_series': portfolio_series, 'benchmark_price_series': benchmark_price_series, 'performance_data': perf_data, 'results_df': results_df, 'actual_calc_end_ts': actual_calc_end_ts, 'safe_start_date': pd.to_datetime(df_close_full.loc[safe_start_date:safe_viz_end_date].index.min()), 'safe_viz_end_date': pd.to_datetime(df_close_full.loc[safe_start_date:safe_viz_end_date].index.max()), 'error': None }

# --- C. THE UI WRAPPER ---

def plot_walk_forward_analyzer(df_ohlcv, 
                               default_start_date=None, default_calc_period='3M', default_fwd_period='1M',
                               default_metric='Sharpe (ATR)', default_rank_start=1, default_rank_end=10,
                               default_benchmark_ticker='VOO'):
    print("Initializing Walk-Forward Analyzer...")
    if not isinstance(df_ohlcv.index, pd.MultiIndex): raise ValueError("Input DataFrame must have a (Ticker, Date) MultiIndex.")
    df_ohlcv = df_ohlcv.sort_index()
    print("Pre-processing data (unstacking)...")
    df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
    df_high_full = df_ohlcv['Adj High'].unstack(level=0)
    df_low_full = df_ohlcv['Adj Low'].unstack(level=0)
    
    start_date_picker = widgets.DatePicker(description='Start Date:', value=pd.to_datetime(default_start_date), disabled=False)
    calc_period_options = {'1M': pd.DateOffset(months=1), '3M': pd.DateOffset(months=3), '6M': pd.DateOffset(months=6), '1Y': pd.DateOffset(years=1)}
    fwd_period_options = {'0D': pd.DateOffset(days=0), '1W': pd.DateOffset(weeks=1), '2W': pd.DateOffset(weeks=2), '1M': pd.DateOffset(months=1), '3M': pd.DateOffset(months=3)}
    calc_period_dropdown = widgets.Dropdown(options=calc_period_options.keys(), value=default_calc_period, description='Calc Period:')
    fwd_period_dropdown = widgets.Dropdown(options=fwd_period_options.keys(), value=default_fwd_period, description='Fwd Period:')
    metrics = ['Price', 'Sharpe', 'Sharpe (ATR)']
    metric_dropdown = widgets.Dropdown(options=metrics, value=default_metric, description='Metric:')
    rank_options = [1, 5, 10, 20, 30, 40, 50, 75, 100]
    rank_start_dropdown = widgets.Dropdown(options=rank_options, value=default_rank_start, description='Rank Start:')
    rank_end_dropdown = widgets.Dropdown(options=rank_options, value=default_rank_end, description='Rank End:')
    benchmark_ticker_input = widgets.Text(value=default_benchmark_ticker, description='Benchmark:', placeholder='Enter Ticker')
    update_button = widgets.Button(description="Update Chart", button_style='primary')
    ticker_list_output = widgets.Output()
    results_container = [None]
    
    fig = go.FigureWidget()
    max_traces = 50
    for i in range(max_traces): fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name=f'placeholder_{i}', visible=False, showlegend=False))
    fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name='Benchmark', visible=True, showlegend=True, line=dict(color='black', width=3, dash='dash')))
    fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name='Group Portfolio', visible=True, showlegend=True, line=dict(color='green', width=3)))

    def update_plot(button_click):
        ticker_list_output.clear_output()
        start_date = pd.to_datetime(start_date_picker.value)
        calc_period = calc_period_options[calc_period_dropdown.value]; fwd_period = fwd_period_options[fwd_period_dropdown.value]
        metric = metric_dropdown.value; rank_start, rank_end = rank_start_dropdown.value, rank_end_dropdown.value
        benchmark_ticker = benchmark_ticker_input.value.strip().upper()
        if rank_start > rank_end:
            with ticker_list_output: print("Error: 'Rank Start' must be <= 'Rank End'."); return

        results = run_walk_forward_step(df_close_full, df_high_full, df_low_full, start_date, calc_period, fwd_period, metric, rank_start, rank_end, benchmark_ticker)
        
        if results['error']:
            with ticker_list_output: print(f"Error: {results['error']}")
            return
            
        with fig.batch_update():
            for i in range(max_traces):
                trace = fig.data[i]
                if i < len(results['tickers_to_display']):
                    ticker = results['tickers_to_display'][i]
                    trace.x, trace.y, trace.name = results['normalized_plot_data'].index, results['normalized_plot_data'][ticker], ticker
                    trace.visible, trace.showlegend = True, True
                else: trace.visible, trace.showlegend = False, False
            benchmark_trace = fig.data[max_traces]
            if results['benchmark_price_series'] is not None and not results['benchmark_price_series'].loc[results['safe_start_date']:results['safe_viz_end_date']].dropna().empty:
                normalized_benchmark = results['benchmark_price_series'].loc[results['safe_start_date']:results['safe_viz_end_date']] / results['benchmark_price_series'].loc[results['safe_start_date']:].bfill().iloc[0]
                benchmark_trace.x, benchmark_trace.y = normalized_benchmark.index, normalized_benchmark
                benchmark_trace.name = f"Benchmark ({benchmark_ticker})"; benchmark_trace.visible = True
            else: benchmark_trace.visible = False
            portfolio_trace = fig.data[max_traces + 1]
            portfolio_trace.x, portfolio_trace.y = results['portfolio_series'].index, results['portfolio_series']
            portfolio_trace.name = 'Group Portfolio'; portfolio_trace.visible = True
            fig.layout.shapes = []; fig.add_shape(type="line", x0=results['actual_calc_end_ts'], y0=0, x1=results['actual_calc_end_ts'], y1=1, xref='x', yref='paper', line=dict(color="grey", width=2, dash="dash"))
            
        results_container[0] = results['results_df']
        
        with ticker_list_output:
            print(f"Analyzing from {results['safe_start_date'].date()} to {results['safe_viz_end_date'].date()}.")
            print(f"  - Ranking based on performance from {results['safe_start_date'].date()} to {results['actual_calc_end_ts'].date()}.")
            pprint.pprint(results['tickers_to_display'], width=120, compact=True)
            
            p = results['performance_data']
            rows = []
            rows.append({'Metric': 'Group Portfolio Gain', 'Full': p['full_p_gain'], 'Calc': p['calc_p_gain'], 'Fwd': p['fwd_p_gain']})
            if not np.isnan(p['full_b_gain']):
                rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Gain', 'Full': p['full_b_gain'], 'Calc': p['calc_b_gain'], 'Fwd': p['fwd_b_gain']})
                rows.append({'Metric': 'Gain Delta (vs Bm)', 'Full': p['full_p_gain'] - p['full_b_gain'], 'Calc': p['calc_p_gain'] - p['calc_b_gain'], 'Fwd': p['fwd_p_gain'] - p['fwd_b_gain']})
            rows.append({'Metric': 'Group Portfolio Sharpe', 'Full': p['full_p_sharpe'], 'Calc': p['calc_p_sharpe'], 'Fwd': p['fwd_p_sharpe']})
            if not np.isnan(p['full_b_sharpe']):
                rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Sharpe', 'Full': p['full_b_sharpe'], 'Calc': p['calc_b_sharpe'], 'Fwd': p['fwd_b_sharpe']})
                rows.append({'Metric': 'Sharpe Delta (vs Bm)', 'Full': p['full_p_sharpe'] - p['full_b_sharpe'], 'Calc': p['calc_p_sharpe'] - p['calc_b_sharpe'], 'Fwd': p['fwd_p_sharpe'] - p['fwd_b_sharpe']})
            report_df = pd.DataFrame(rows).set_index('Metric')
            gain_rows = [row for row in report_df.index if 'Gain' in row or 'Delta' in row]
            sharpe_rows = [row for row in report_df.index if 'Sharpe' in row]
            styled_df = report_df.style.format('{:+.2%}', na_rep='N/A', subset=(gain_rows, report_df.columns)).format('{:+.2f}', na_rep='N/A', subset=(sharpe_rows, report_df.columns)).set_properties(**{'text-align': 'right', 'width': '100px'}).set_table_styles([{'selector': 'th.col_heading', 'props': [('text-align', 'right')]}, {'selector': 'th.row_heading', 'props': [('text-align', 'left')]}])
            print("\n--- Strategy Performance Summary ---")
            display(styled_df)
            
    fig.update_layout(title_text='Walk-Forward Performance Analysis', xaxis_title='Date', yaxis_title='Normalized Price (Start = 1)', hovermode='x unified', legend_title_text='Tickers (Ranked)', height=700, margin=dict(t=50))
    fig.add_hline(y=1, line_width=1, line_dash="dash", line_color="grey")
    update_button.on_click(update_plot)
    controls_row1 = widgets.HBox([start_date_picker, calc_period_dropdown, fwd_period_dropdown])
    controls_row2 = widgets.HBox([metric_dropdown, rank_start_dropdown, rank_end_dropdown, benchmark_ticker_input, update_button])
    ui_container = widgets.VBox([controls_row1, controls_row2, ticker_list_output], layout=widgets.Layout(margin='10px 0 20px 0'))
    display(ui_container, fig)
    update_plot(None)
    return results_container

# --- D. VERIFICATION TOOLS ---

def verify_group_tickers_walk_forward_calculation(df_ohlcv, tickers_to_verify, benchmark_ticker,
                                                  start_date, calc_period, fwd_period, export_csv=False):
    # This function is unchanged
    display(Markdown(f"## Verification Report for Portfolio vs. Benchmark"))
    display(Markdown(f"**Portfolio Tickers:** `{tickers_to_verify}`"))
    display(Markdown(f"**Benchmark Ticker:** `{benchmark_ticker}`"))
    period_options = { '1M': pd.DateOffset(months=1), '3M': pd.DateOffset(months=3), '6M': pd.DateOffset(months=6), '1Y': pd.DateOffset(years=1), '0D': pd.DateOffset(days=0), '1W': pd.DateOffset(weeks=1), '2W': pd.DateOffset(weeks=2) }
    df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
    start_date_ts = pd.to_datetime(start_date)
    calc_offset = period_options[calc_period]; fwd_offset = period_options[fwd_period]
    calc_end_date_ts_theoretical = start_date_ts + calc_offset
    fwd_end_date_ts_theoretical = calc_end_date_ts_theoretical + fwd_offset
    actual_calc_end_ts = df_close_full.loc[start_date_ts:calc_end_date_ts_theoretical].index.max()
    display(Markdown(f"**Analysis Start Date:** `{start_date_ts.date()}`"))
    display(Markdown(f"**Calculation Period End Date:** `{actual_calc_end_ts.date()}`"))
    display(Markdown(f"**Forward Period End Date:** `{fwd_end_date_ts_theoretical.date()}`"))
    portfolio_prices_raw_slice = df_close_full[tickers_to_verify].loc[start_date_ts:fwd_end_date_ts_theoretical]
    normalized_portfolio_prices = portfolio_prices_raw_slice.div(portfolio_prices_raw_slice.bfill().iloc[0])
    portfolio_value_series = normalized_portfolio_prices.mean(axis=1)
    try: benchmark_price_series = df_close_full[benchmark_ticker]
    except KeyError as e: print(f"---! ERROR: Ticker {e} not found !---"); return

    def print_verification_steps(title, price_series):
        display(Markdown(f"#### Verification for: `{title}`"))
        if price_series.dropna().shape[0] < 2: print("  - Not enough data points."); return {'gain': np.nan, 'sharpe': np.nan}
        start_price = price_series.bfill().iloc[0]; end_price = price_series.ffill().iloc[-1]
        gain = (end_price / start_price) - 1
        print(f"  - Start Value (on {price_series.first_valid_index().date()}): {start_price:,.4f}\n  - End Value   (on {price_series.last_valid_index().date()}): {end_price:,.4f}\n  - Gain = ({end_price:,.4f} / {start_price:,.4f}) - 1 = {gain:.2%}")
        returns = price_series.pct_change()
        mean_return = returns.mean(); std_return = returns.std()
        sharpe = (mean_return / std_return * np.sqrt(252)) if std_return > 0 and std_return != np.inf else np.nan
        print(f"\n  - Mean Daily Return: {mean_return:.6f}\n  - Std Dev of Daily Return: {std_return:.6f}\n  - Sharpe = ({mean_return:.6f} / {std_return:.6f}) * sqrt(252) = {sharpe:.2f}")
        return {'gain': gain, 'sharpe': sharpe}

    display(Markdown("### A. Calculation Period Analysis ('In-Sample')"))
    perf_calc_p = print_verification_steps("Group Portfolio", portfolio_value_series.loc[start_date_ts:actual_calc_end_ts])
    perf_calc_b = print_verification_steps(f"Benchmark ({benchmark_ticker})", benchmark_price_series.loc[start_date_ts:actual_calc_end_ts])
    display(Markdown("\n### B. Forward Period Analysis ('Moment of Truth')"))
    perf_fwd_p = print_verification_steps("Group Portfolio", portfolio_value_series.loc[actual_calc_end_ts:fwd_end_date_ts_theoretical])
    perf_fwd_b = print_verification_steps(f"Benchmark ({benchmark_ticker})", benchmark_price_series.loc[actual_calc_end_ts:fwd_end_date_ts_theoretical])
    display(Markdown("\n### C. Full Period Analysis (Total)"))
    perf_full_p = print_verification_steps("Group Portfolio", portfolio_value_series)
    perf_full_b = print_verification_steps(f"Benchmark ({benchmark_ticker})", benchmark_price_series.loc[start_date_ts:fwd_end_date_ts_theoretical])
    display(Markdown("\n### D. Final Summary Table (matches analyzer output)"))
    rows = []
    rows.append({'Metric': 'Group Portfolio Gain', 'Full': perf_full_p['gain'], 'Calc': perf_calc_p['gain'], 'Fwd': perf_fwd_p['gain']})
    rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Gain', 'Full': perf_full_b['gain'], 'Calc': perf_calc_b['gain'], 'Fwd': perf_fwd_b['gain']})
    rows.append({'Metric': 'Gain Delta (vs Bm)', 'Full': perf_full_p['gain'] - perf_full_b['gain'], 'Calc': perf_calc_p['gain'] - perf_calc_b['gain'], 'Fwd': perf_fwd_p['gain'] - perf_fwd_b['gain']})
    rows.append({'Metric': 'Group Portfolio Sharpe', 'Full': perf_full_p['sharpe'], 'Calc': perf_calc_p['sharpe'], 'Fwd': perf_fwd_p['sharpe']})
    rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Sharpe', 'Full': perf_full_b['sharpe'], 'Calc': perf_calc_b['sharpe'], 'Fwd': perf_fwd_b['sharpe']})
    rows.append({'Metric': 'Sharpe Delta (vs Bm)', 'Full': perf_full_p['sharpe'] - perf_full_b['sharpe'], 'Calc': perf_calc_p['sharpe'] - perf_calc_b['sharpe'], 'Fwd': perf_fwd_p['sharpe'] - perf_fwd_b['sharpe']})
    report_df = pd.DataFrame(rows).set_index('Metric')
    gain_rows = [row for row in report_df.index if 'Gain' in row or 'Delta' in row]
    sharpe_rows = [row for row in report_df.index if 'Sharpe' in row]
    styled_df = report_df.style.format('{:+.2%}', na_rep='N/A', subset=(gain_rows, report_df.columns)).format('{:+.2f}', na_rep='N/A', subset=(sharpe_rows, report_df.columns)).set_properties(**{'text-align': 'right', 'width': '100px'}).set_table_styles([{'selector': 'th.col_heading', 'props': [('text-align', 'right')]}, {'selector': 'th.row_heading', 'props': [('text-align', 'left')]}])
    display(styled_df)
    if export_csv:
        export_df = pd.DataFrame({'Portfolio_Value_Normalized': portfolio_value_series, 'Portfolio_Return': portfolio_value_series.pct_change(), f'Benchmark_Price_{benchmark_ticker}': benchmark_price_series})
        filename = f"verification_group_tickers_{start_date_ts.strftime('%Y%m%d')}.csv"
        export_df.to_csv(filename, float_format='%.6f')
        print(f"\n✅ Detailed group verification data exported to '{filename}'")

def verify_ticker_ranking_metrics(df_ohlcv, 
                                  ticker, 
                                  start_date, 
                                  calc_period, 
                                  fwd_period, 
                                  export_csv=False):
    display(Markdown(f"## Verification Report for Ticker Ranking: `{ticker}`"))
    period_options = { '1M': pd.DateOffset(months=1), '3M': pd.DateOffset(months=3), '6M': pd.DateOffset(months=6), '1Y': pd.DateOffset(years=1), '0D': pd.DateOffset(days=0), '1W': pd.DateOffset(weeks=1), '2W': pd.DateOffset(weeks=2) }
    try: df_ticker = df_ohlcv.loc[ticker].sort_index()
    except KeyError: print(f"---! ERROR: Ticker '{ticker}' not found !---"); return
    start_date_ts = pd.to_datetime(start_date)
    calc_offset = period_options[calc_period]; fwd_offset = period_options[fwd_period]
    calc_end_date_ts = start_date_ts + calc_offset; fwd_end_date_ts = calc_end_date_ts + fwd_offset
    display(Markdown(f"**Analysis Start Date:** `{start_date_ts.date()}`"))
    display(Markdown(f"**Requested Calculation Period:** `{start_date_ts.date()}` to `{calc_end_date_ts.date()}`"))
    display(Markdown(f"**Requested Forward Period:**   `{calc_end_date_ts.date()}` to `{fwd_end_date_ts.date()}`"))
    display(Markdown("### A. Calculation Period Analysis (for Ranking Metrics)"))
    calc_df = df_ticker.loc[start_date_ts:calc_end_date_ts].copy()
    if calc_df['Adj Close'].notna().sum() < 2: print("\n---! ERROR: Not enough data points !---"); return
    actual_calc_end_date = calc_df.index.max().date()
    display(Markdown(f"**Actual Dates Used:** `{calc_df.index.min().date()}` to `{actual_calc_end_date}`"))
    calc_gain = calculate_gain(calc_df['Adj Close'])
    calc_start_price = calc_df['Adj Close'].bfill().iloc[0]
    calc_end_price = calc_df['Adj Close'].ffill().iloc[-1]
    display(Markdown("#### `CalcGain` Verification:"))
    print(f"  - Calc Start Price: ${calc_start_price:.2f}\n  - Calc End Price:   ${calc_end_price:.2f}  <-- 'CalcPrice'\n  - CalcGain = {calc_gain:.2%}")
    display(Markdown("#### `MetricValue` Verification:"))
    price_metric = (calc_end_price / calc_start_price)
    print(f"\n1. Price Metric:\n   - Formula: Last Price / First Price = {price_metric:.4f}")
    daily_returns = calc_df['Adj Close'].bfill().ffill().pct_change()
    sharpe_ratio = calculate_sharpe(daily_returns)
    print(f"\n2. Sharpe Metric:\n   - Mean Daily Return: {daily_returns.mean():.6f}\n   - Std Dev Daily Return: {daily_returns.std():.6f}\n   - Annualized Sharpe = {sharpe_ratio:.4f}")
    print(f"\n3. Sharpe (ATR) Metric:")
    tr = np.maximum(calc_df['Adj High'] - calc_df['Adj Low'], abs(calc_df['Adj High'] - calc_df['Adj Close'].shift(1)), abs(calc_df['Adj Low'] - calc_df['Adj Close'].shift(1)))
    atr = tr.ewm(alpha=1/14, adjust=False).mean()
    atrp_series = atr / calc_df['Adj Close']
    atrp_mean = atrp_series.mean()
    sharpe_atr = (daily_returns.mean() / atrp_mean) if atrp_mean > 0 else 0
    print(f"   - Mean Daily Return: {daily_returns.mean():.6f} (same as above)\n   - Average ATR Percent (ATRP): {atrp_mean:.6f}\n   - Sharpe (ATR) = {sharpe_atr:.4f}")
    display(Markdown("\n### B. Forward Period Analysis (`FwdGain`)"))
    fwd_df = df_ticker.loc[actual_calc_end_date:fwd_end_date_ts].copy()
    fwd_gain = calculate_gain(fwd_df['Adj Close'])
    fwd_end_price = fwd_df['Adj Close'].ffill().iloc[-1] if fwd_gain is not np.nan else calc_end_price
    print(f"  - Fwd Start Price (Calc End Price): ${calc_end_price:.2f}\n  - Fwd End Price: ${fwd_end_price:.2f}\n  - FwdGain = {fwd_gain:.2%}")
    display(Markdown("\n### C. Final Summary Table"))
    summary_data = {'Metric': ['Price', 'Sharpe', 'Sharpe (ATR)'], 'Calculated Value': [f"{price_metric:.4f}", f"{sharpe_ratio:.4f}", f"{sharpe_atr:.4f}"], 'Corresponds To': ['`MetricValue`', '`MetricValue`', '`MetricValue`'], '---': ['---','---','---'], 'Gain Metric': ['Calc Period Gain', 'Forward Period Gain'], 'Gain Value': [f"{calc_gain:.2%}", f"{fwd_gain:.2%}"], 'Gain Corresponds To': ['`CalcGain`', '`FwdGain`']}
    summary_df = pd.DataFrame(summary_data)
    display(summary_df.style.hide(axis="index"))
    
    if export_csv:
        calc_df['Period'] = 'Calculation'; calc_df['Daily_Return'] = daily_returns; calc_df['True_Range'] = tr; calc_df['ATR_14'] = atr; calc_df['ATRP'] = atrp_series
        fwd_df['Period'] = 'Forward'
        combined_df = pd.concat([calc_df, fwd_df.iloc[1:]])
        filename = f"verification_ticker_{ticker}_{start_date_ts.strftime('%Y%m%d')}.csv"
        combined_df.to_csv(filename, float_format='%.6f')
        print(f"\n✅ Detailed ticker data exported to '{filename}'")


# --- 1. SAMPLE DATA (for reproducibility) ---
date_rng = pd.date_range(start='2023-01-01', end='2023-10-31', freq='B')
tickers = ['CVNA', 'VRT', 'APP', 'SMCI', 'IONQ', 'XPO', 'XP', 'AD', 'USM', 'MOD', 'VOO', 'QQQ']
data = []
np.random.seed(42) # for reproducibility
for ticker in tickers:
    price = 100 + (np.random.randn(len(date_rng)).cumsum() * (0.5 if ticker != 'SMCI' else 2.5))
    high = price + np.random.uniform(0, 2, size=len(date_rng))
    low = price - np.random.uniform(0, 2, size=len(date_rng))
    open_price = price + np.random.uniform(-1, 1, size=len(date_rng))
    volume = np.random.randint(100000, 5000000, size=len(date_rng))
    ticker_df = pd.DataFrame({'Date': date_rng,'Ticker': ticker,'Adj Open': open_price,'Adj High': high,'Adj Low': low,'Adj Close': price,'Volume': volume})
    data.append(ticker_df)
df_full = pd.concat(data)
df_OHLCV = df_full.set_index(['Ticker', 'Date'])

# --- 2. TEST EXECUTION ---
test_start_date = '2023-04-01'
test_calc_period = '6M'
test_fwd_period = '2W'
test_metric = 'Price'
test_rank_start = 1
test_rank_end = 10
test_benchmark = 'VOO'

print("--- RUNNING REFACTORED CODE TEST ---")
walk_forward_results = plot_walk_forward_analyzer(
    df_OHLCV,
    default_start_date=test_start_date,
    default_calc_period=test_calc_period,
    default_fwd_period=test_fwd_period,
    default_metric=test_metric,
    default_rank_start=test_rank_start,
    default_rank_end=test_rank_end,
    default_benchmark_ticker=test_benchmark
)
print("\n--- TEST COMPLETE ---")

--- RUNNING REFACTORED CODE TEST ---
Initializing Walk-Forward Analyzer...
Pre-processing data (unstacking)...


FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'placeholder_0',
              'showlegend': False,
              'type': 'scatter',
              'uid': '14291851-4c45-49b4-8300-1e3b3dc98009',
              'visible': False,
              'x': [None],
              'y': [None]},
             {'mode': 'lines',
              'name': 'placeholder_1',
              'showlegend': False,
              'type': 'scatter',
              'uid': '7162b82b-c9e8-42f7-aaa1-211dba496ea4',
              'visible': False,
              'x': [None],
              'y': [None]},
             {'mode': 'lines',
              'name': 'placeholder_2',
              'showlegend': False,
              'type': 'scatter',
              'uid': '954dde29-6922-4753-837e-06bd1a594ed3',
              'visible': False,
              'x': [None],
              'y': [None]},
             {'mode': 'lines',
              'name': 'placeholder_3',
              'showlegend': False,
              'type': 


--- TEST COMPLETE ---


In [6]:
download_path = Path.home() / "Downloads"  
# OHLCV_file_path = r'c:\Users\ping\Files_win10\python\py311\stocks\data\df_OHLCV_clean_stocks_etfs.parquet'
OHLCV_file_path = r'c:\Users\ping\Files_win10\python\py311\stocks\data\df_OHLCV_stocks_etfs.parquet'

df_OHLCV = pd.read_parquet(OHLCV_file_path, engine='pyarrow')
print(f'df_OHLCV.info() :\n{df_OHLCV.info()}')
print(f'\ndf_OHLCV.head():\n{df_OHLCV.head()}')
print(f'\ndf_OHLCV.tail():\n{df_OHLCV.tail()}')

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2115382 entries, ('A', Timestamp('2020-01-02 00:00:00')) to ('ZWS', Timestamp('2025-10-03 00:00:00'))
Data columns (total 5 columns):
 #   Column     Dtype  
---  ------     -----  
 0   Adj Open   float64
 1   Adj High   float64
 2   Adj Low    float64
 3   Adj Close  float64
 4   Volume     int64  
dtypes: float64(4), int64(1)
memory usage: 88.9+ MB
df_OHLCV.info() :
None

df_OHLCV.head():
                   Adj Open  Adj High  Adj Low  Adj Close   Volume
Ticker Date                                                       
A      2020-01-02   82.4973   82.9295  81.8250    82.5453  1468677
       2020-01-03   81.3160   81.9499  81.1527    81.2200  1164425
       2020-01-06   80.6725   81.4600  80.2884    81.4600  2075412
       2020-01-07   80.6341   81.8826  80.6149    81.7098  1754187
       2020-01-08   82.5549   83.0447  81.8250    82.5165  1923806

df_OHLCV.tail():
                   Adj Open  Adj High  Adj Low  Adj Close  Volume
Ti

In [8]:
# --- B. DYNAMIC DATA QUALITY FILTER FUNCTIONS ---
# FINAL RECOMMENDED VERSION v2
def calculate_rolling_quality_metrics(df_ohlcv, window=252, min_periods=126, debug=False):
    """
    Calculates rolling quality metrics for OHLCV data to identify tradable tickers.

    This function enriches the input DataFrame with metrics that quantify data
    quality and liquidity over a specified rolling window.

    Args:
        df_ohlcv (pd.DataFrame): DataFrame with a ('Ticker', 'Date') MultiIndex
                                 and columns for OHLCV data.
        window (int): The lookback period in days for the rolling calculations.
                      Defaults to 252 (approx. one trading year).
        min_periods (int): The minimum number of observations in the window required
                           to have a value. Defaults to 126 (approx. half a year).
        debug (bool): If True, returns a DataFrame with all intermediate
                      calculations. Defaults to False, returning only the
                      final quality metrics.

    Returns:
        pd.DataFrame: A DataFrame containing the calculated quality metrics. See the
                      'Metrics Description' section below for column details. If debug is True,
                      the output will also include original and intermediate columns.

    Metrics Description:
        RollingStalePct (float):
            The rolling percentage (0.0 to 1.0) of days considered 'stale' within
            the lookback window. A day is flagged as stale if its trading volume is
            zero OR its high price is equal to its low price. This metric helps
            identify non-trading assets or assets with poor quality data feeds.
            **A lower value is better.**

        RollingMedianVolume (float):
            The rolling median of the daily dollar volume (Adj Close * Volume). The
            median is used to provide a robust measure of typical liquidity that is
            insensitive to single-day volume spikes (outliers). This metric is crucial
            for filtering out illiquid stocks where trades could incur significant
            slippage.
            **A higher value is better.**

        RollingSameVolCount (float):
            A rolling count of the number of times a day's volume was exactly equal
            to the previous day's volume. This is a heuristic for detecting
            potential low-quality data feeds, as this event is statistically rare
            for actively traded assets and may indicate improper forward-filling of
            missing data.
            **A lower value is better.**
    """
    print(f"--- Calculating Rolling Quality Metrics (Window: {window} days) ---")
    df = df_ohlcv.copy()

    # Improvement 1: Ensure data is sorted for correctness
    if not df.index.is_monotonic_increasing:
        print("ℹ️ Data is not sorted. Sorting index chronologically...")
        df.sort_index(inplace=True)

    # --- Intermediate calculations ---
    # This calculation is always the same, whether in debug mode or not.
    df['IsStale'] = np.where((df['Volume'] == 0) | (df['Adj High'] == df['Adj Low']), 1, 0)
    df['DollarVolume'] = df['Adj Close'] * df['Volume']
    df['HasSameVolumeAsPrevDay'] = (df.groupby(level='Ticker')['Volume'].diff() == 0).astype(int)
    
    # === NEW: Add component columns ONLY if in debug mode ===
    if debug:
        print("...Adding debug component columns for 'IsStale'.")
        df['Debug_HasZeroVolume'] = (df['Volume'] == 0).astype(int)
        df['Debug_IsHighEqLow'] = (df['Adj High'] == df['Adj Low']).astype(int)
    # =========================================================

    # --- Rolling calculations ---
    grouped = df.groupby(level='Ticker')
    stale_pct = grouped['IsStale'].rolling(window=window, min_periods=min_periods).mean()
    median_vol = grouped['DollarVolume'].rolling(window=window, min_periods=min_periods).median()
    same_vol_count = grouped['HasSameVolumeAsPrevDay'].rolling(window=window, min_periods=min_periods).sum()

    quality_df = pd.concat([stale_pct, median_vol, same_vol_count], axis=1)
    quality_df.columns = ['RollingStalePct', 'RollingMedianVolume', 'RollingSameVolCount']
    quality_df.index = quality_df.index.droplevel(0)

    print("✅ Rolling metrics calculation complete.")
    
    if debug:
        # For debugging, return the original data joined with all calculations
        print("...Debug mode enabled, returning full calculation trace.")
        # The 'df' DataFrame now contains the extra debug columns, which will be included automatically.
        full_df = df.join(quality_df)
        return full_df
    else:
        # Default production behavior
        return quality_df


def get_eligible_universe(quality_metrics_df, filter_date, thresholds):
    """
    Filters tickers to create an eligible universe for a specific date based on quality metrics.

    Args:
        quality_metrics_df (pd.DataFrame): The output from calculate_rolling_quality_metrics.
        filter_date (pd.Timestamp or str): The specific date to perform the filtering on.
        thresholds (dict): A dictionary with the filtering rules, e.g.,
                           {'min_median_dollar_volume': 1e6, 'max_stale_pct': 0.05, 'max_same_vol_count': 1}.

    Returns:
        list: A list of ticker symbols that are eligible on the filter_date.
    """
    try:
        metrics_on_date = quality_metrics_df.xs(pd.to_datetime(filter_date), level='Date')
    except KeyError:
        print(f"Warning: Filter date {pd.to_datetime(filter_date).date()} not found in quality metrics index. Returning all tickers.")
        return quality_metrics_df.index.get_level_values('Ticker').unique().tolist()

    mask = (
        (metrics_on_date['RollingMedianVolume'] >= thresholds['min_median_dollar_volume']) &
        (metrics_on_date['RollingStalePct'] <= thresholds['max_stale_pct']) &
        (metrics_on_date['RollingSameVolCount'] <= thresholds['max_same_vol_count'])
    )
    
    eligible_tickers = metrics_on_date[mask].index.tolist()
    all_tickers = metrics_on_date.index.tolist()
    
    print(f"Dynamic Filter ({pd.to_datetime(filter_date).date()}): Kept {len(eligible_tickers)} of {len(all_tickers)} tickers.")
    return eligible_tickers    


In [9]:
# Our inspection rules
my_thresholds = {
    'min_median_dollar_volume': 10_600_000, # Must have high liquidity
    'max_stale_pct': 0.1,                   # Allow very few stale days (max 10%)
    'max_same_vol_count': 1                 # Allow at most 1 suspicious volume event
}

In [10]:
quality_df = calculate_rolling_quality_metrics(
    df_ohlcv=df_OHLCV,
    window=252,
    min_periods=126,
    # debug=True  # <-- The key to our new, improved workflow    
)

# Let's look at the input for our next function (a snippet from the last day)
print("--- Quality Metrics for 2025-10-03 (Last Day) ---")
print(quality_df.xs('2025-10-03', level='Date'))


--- Calculating Rolling Quality Metrics (Window: 252 days) ---
✅ Rolling metrics calculation complete.
--- Quality Metrics for 2025-10-03 (Last Day) ---
        RollingStalePct  RollingMedianVolume  RollingSameVolCount
Ticker                                                           
A                   0.0         2.184756e+08                  0.0
AA                  0.0         1.724810e+08                  0.0
AAL                 0.0         6.185211e+08                  0.0
AAON                0.0         7.029373e+07                  0.0
AAPL                0.0         1.056353e+10                  0.0
...                 ...                  ...                  ...
ZM                  0.0         1.973018e+08                  0.0
ZS                  0.0         3.879914e+08                  0.0
ZTO                 0.0         4.067204e+07                  0.0
ZTS                 0.0         4.400619e+08                  0.0
ZWS                 0.0         3.283692e+07           

In [11]:
eligible_tickers = get_eligible_universe(quality_metrics_df=quality_df, 
                                         filter_date='2021-06-01', 
                                         thresholds=my_thresholds)

Dynamic Filter (2021-06-01): Kept 1229 of 1408 tickers.


In [16]:
print(f'eligible_tickers: {eligible_tickers}') 

eligible_tickers: ['A', 'AA', 'AAL', 'AAPL', 'ABBV', 'ABEV', 'ABT', 'ACGL', 'ACI', 'ACM', 'ACN', 'ACWI', 'ACWX', 'ADBE', 'ADC', 'ADI', 'ADM', 'ADP', 'ADSK', 'ADT', 'AEE', 'AEIS', 'AEM', 'AEP', 'AER', 'AES', 'AFG', 'AFL', 'AGCO', 'AGG', 'AGI', 'AGNC', 'AIG', 'AIT', 'AIZ', 'AJG', 'AKAM', 'AL', 'ALB', 'ALC', 'ALGN', 'ALK', 'ALL', 'ALLE', 'ALLY', 'ALNY', 'ALSN', 'ALV', 'AM', 'AMAT', 'AMCR', 'AMD', 'AME', 'AMG', 'AMGN', 'AMH', 'AMKR', 'AMLP', 'AMP', 'AMT', 'AMX', 'AMZN', 'AN', 'ANET', 'AON', 'AOS', 'APA', 'APD', 'APG', 'APH', 'APO', 'APPF', 'APTV', 'AR', 'ARCC', 'ARE', 'ARES', 'ARGX', 'ARKK', 'ARMK', 'ASML', 'ASND', 'ATI', 'ATO', 'AU', 'AVAV', 'AVB', 'AVGO', 'AVTR', 'AVY', 'AWI', 'AWK', 'AXON', 'AXP', 'AXS', 'AXTA', 'AYI', 'AZN', 'AZO', 'B', 'BA', 'BABA', 'BAC', 'BAH', 'BALL', 'BAP', 'BAX', 'BBD', 'BBEU', 'BBIO', 'BBJP', 'BBVA', 'BBY', 'BCE', 'BCS', 'BDX', 'BE', 'BEKE', 'BEN', 'BEP', 'BF-B', 'BG', 'BHP', 'BIDU', 'BIIB', 'BIL', 'BILI', 'BIO', 'BIP', 'BIV', 'BJ', 'BK', 'BKLN', 'BKNG', 'BKR', 